In [ ]:
import sys
home_path = '/home/cfpb/gilliesi/bisg-py'
sys.path.insert(1, home_path+'/utils')
from LookupTables import LookUpTables
from BISG import BISG
from DataFrameUtils import input_to_dataframe
import pandas as pd 
import numpy as np 

In [ ]:
MyBISG= BISG(census_surname_data= [home_path+'/input_files/Names_2010Census.csv',home_path+'/input_files/app_c.csv'],
                census_block_group_data=home_path+'/input_files/blkgrp_over_18_race_jan20.csv',
                census_tract_data=home_path+'/input_files/tract_over_18_race_jan20.csv',
                census_zip_data=home_path+'/input_files/zip_over_18_race_jan20.csv')

# Surname Test

In [ ]:
census_surnames_stata=input_to_dataframe(home_path+'/test_files/census_surnames_lower.dta')

In [ ]:
renamer={'pct'+race : 'pr_'+ race+'_given_surname' for race in MyBISG.LookupTables.race_list}
renamer['pct2prace']='pr_'+ 'mult_other'+'_given_surname'
del renamer['pctmult_other']

census_surname_lookup_stata = census_surnames_stata.rename(columns=renamer)[['name']+list(renamer.values())].sort_values(by='name').reset_index(drop=True)
census_surname_lookup_python = MyBISG.LookupTables.pr_race_given_surname[['name']+list(renamer.values())].sort_values(by='name').reset_index(drop=True)

In [ ]:
(census_surname_lookup_stata==census_surname_lookup_python).all(axis=1)[(census_surname_lookup_stata==census_surname_lookup_python).all(axis=1)==False]
(census_surname_lookup_stata==census_surname_lookup_python).all().all()

# Geo Tests

In [ ]:
race_list=MyBISG.LookupTables.race_list
geo='zip'
census_geo_python=MyBISG.LookupTables.pr_zip_given_race
census_geo_stata=input_to_dataframe(home_path+'/test_files/zip_attr_over18_jan20.dta')

#rename the columns
renamer=({f'geo_pr_{race}' : f'pr_{race}_given_geo' for race in race_list})
d2={f'here_given_{race}':f'pr_geo_given_{race}' for race in race_list }
renamer['here']='pr_geo'
renamer.update(d2)

census_geo_stata=census_geo_stata.rename(columns=renamer)[['GeoInd']+list(renamer.values())].sort_values('GeoInd').reset_index(drop=True)
census_geo_python=census_geo_python[['GeoInd']+list(renamer.values())].sort_values('GeoInd').reset_index(drop=True)

((census_geo_stata.drop('GeoInd',axis=1)-census_geo_python.drop('GeoInd',axis=1)).abs()>=10e-5)[((census_geo_stata.drop('GeoInd',axis=1)-census_geo_python.drop('GeoInd',axis=1)).abs()>=10e-5).all(axis=1)].shape

In [ ]:
table